In [6]:
from datasets import load_dataset

dataset = load_dataset("imdb", split={"train": "train[:2000]", "test": "test[:500]"})

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

c:\Users\sabie\OneDrive\Documents\Code\Python\micro-scikit\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sabie\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. 

In [8]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 500/500 [00:00<00:00, 2914.84 examples/s]


In [9]:
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results", 
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.000279
2,0.013000,0.000094
3,0.013000,0.000068


c:\Users\sabie\OneDrive\Documents\Code\Python\micro-scikit\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sabie\OneDrive\Documents\Code\Python\micro-scikit\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=750, training_loss=0.00866904636969169, metrics={'train_runtime': 6361.5154, 'train_samples_per_second': 0.943, 'train_steps_per_second': 0.118, 'total_flos': 397402195968000.0, 'train_loss': 0.00866904636969169, 'epoch': 3.0})

In [11]:
trainer.evaluate()

{'eval_loss': 6.762630073353648e-05,
 'eval_runtime': 74.0586,
 'eval_samples_per_second': 6.751,
 'eval_steps_per_second': 0.851,
 'epoch': 3.0}